# Profile Python

https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html summary:

* `%time`: Time the execution of a single statement
* `%timeit`: Time repeated execution of a single statement for more accuracy
* `%prun`: Run code with the profiler
* `%lprun`: Run code with the line-by-line profiler
* `%memit`: Measure the memory use of a single statement
* `%mprun`: Run code with the line-by-line memory profiler

See also <BR>
http://codedelish.com/resources/Python/profiling.html<BR>
http://pynash.org/2013/03/06/timing-and-profiling/

In [5]:
import sys
print(sys.version)

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [5]:
!pip install line-profiler # if you are using a mac, there is an issue https://github.com/rkern/line_profiler/issues/132
!pip install psutil
!pip install memory_profiler

In [4]:
# # if you are using a mac, do the following instead of 
# !git clone https://github.com/rkern/line_profiler.git
# !find line_profiler -name '*.pyx' -exec cython {} \;
# !cd line_profiler && pip install . --user 

Cloning into 'line_profiler'...
remote: Enumerating objects: 348, done.
remote: Total 348 (delta 0), reused 0 (delta 0), pack-reused 348
Receiving objects: 100% (348/348), 154.88 KiB | 3.23 MiB/s, done.
Resolving deltas: 100% (166/166), done.
/Users/msarica/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /Users/msarica/Desktop/UMBC/DS601_Spring21/Week13/notebooks/line_profiler/_line_profiler.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
Processing /Users/msarica/Desktop/UMBC/DS601_Spring21/Week13/notebooks/line_profiler
  Created wheel for line-profiler: filename=line_profiler-2.1.1-cp37-cp37m-macosx_10_9_x86_64.whl size=55093 sha256=5cf26cac3a8adaa1b17fc02bdedda8a639e900c1635175cbd3144695fcc8e215
  Stored in directory: /private/var/folders/lj/kp1zv9lj6nx6pdndvtmq5lcm0000gn/T/pip-ephem-wheel-cache-icgy6f2l/wheels/c3/2b/82/aff44ecf35c94

two functions that we will profile

In [6]:
def mult_by(inpt):
    """
    a simple function that is not complicated
    """
    avalu = inpt*2
    avalu+=1
    return

def primes(n=1000): 
    """
    A more complicated function to be profiled
    
    An underdocumented function with an unclear purpose
    in-line comments added by Ben
    """
    A = [True] * (n+1) # the name of this array is not descriptive :(
    A[0] = False # there is no reasoning provided about why this would be 
    A[1] = False # no reasoning :(
    for i in range(2, int(n**0.5)): # I dislike single-letter variable names
        if A[i]: # again, this is difficult to reason about, both in terms of A and i
            mult_by(i) # not actually relevant to determining a prime number
            for j in range(i**2, n+1, i): # how did the author come up with this?
                A[j] = False

    return [x for x in range(2, n) if A[x]] # for me this is challenging to reason about

# analysis in three parts
1. total time
1. time per function
1. time per line in a specific function

# total time spent in execution using `time` magic function 

see https://ipython.readthedocs.io/en/stable/interactive/magics.html

How long does execution of a function take?

`%time` applies to a single line of code

In [7]:
primes(20) 

[2, 3, 5, 7, 11, 13, 17, 19]

In [8]:
%time primes(20) # call a function and provide an argument

CPU times: user 14 µs, sys: 1e+03 ns, total: 15 µs
Wall time: 17.2 µs


[2, 3, 5, 7, 11, 13, 17, 19]

for "microseconds" (us, 1E-6) versus "milliseconds" (ms, 1E-3), see https://en.wikipedia.org/wiki/Metric_prefix#List_of_SI_prefixes

user versus system versus total: see https://en.wikipedia.org/wiki/CPU_time#Subdivision

We humans care about "wall time"; see https://en.wikipedia.org/wiki/Elapsed_real_time

<BR>
    
Similarly, `%%time` applies to the entire cell

In [9]:
%%time 
primes(20)
primes(30)

CPU times: user 35 µs, sys: 0 ns, total: 35 µs
Wall time: 37.9 µs


[2, 3, 5, 7, 11, 13, 17, 19, 23, 25, 29]

Although the two calls to `primes()` do output text, the display is overwritten by `time`

Displaying output skews measurement results for cell execution, so best practice is to store output to a variable

In [10]:
%time reslt=primes(20)

CPU times: user 15 µs, sys: 1e+03 ns, total: 16 µs
Wall time: 18.8 µs


There's not much difference between times for multiple executions, but the variance increases when the duration is longer

In [11]:
%time reslt=primes(1000000)

print('length of array:',len(reslt))

CPU times: user 154 ms, sys: 5.12 ms, total: 160 ms
Wall time: 158 ms
length of array: 78498


To run multiple timing test, use `%timeit`

In [12]:
%timeit reslt=primes(1000000)

131 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Want to capture the output of `timeit` to a variable? See https://stackoverflow.com/questions/24812253/how-can-i-capture-return-value-with-python-timeit-module

# time spent per function using `cProfile`

https://docs.python.org/3/library/profile.html

In [13]:
import cProfile

In [14]:
cProfile.run('primes(10000000)')

         451 function calls in 1.978 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      446    0.000    0.000    0.000    0.000 <ipython-input-6-2fc7f0ed043d>:1(mult_by)
        1    0.416    0.416    0.416    0.416 <ipython-input-6-2fc7f0ed043d>:25(<listcomp>)
        1    1.534    1.534    1.951    1.951 <ipython-input-6-2fc7f0ed043d>:9(primes)
        1    0.027    0.027    1.978    1.978 <string>:1(<module>)
        1    0.000    0.000    1.978    1.978 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




* `ncalls`:       How many times it was called
* `tottime`:      for the total time spent in the given function (and excluding time made in calls to sub-functions)
* `percall`:      is the quotient of tottime divided by ncalls
* `cumtime`:      is the cumulative time spent in this and all subfunctions (from invocation till exit). This figure is accurate even for recursive functions.
* `percall`:      is the quotient of cumtime divided by primitive calls
* `filename:lineno(function)`:    provides the respective data of each function

# time spent per line within specified function

https://nyu-cds.github.io/python-performance-tuning/03-line_profiler/
    
https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

In [15]:
%load_ext line_profiler

In [16]:
%lprun?

Profile the lines of the `prime` function

In [17]:
%lprun -f primes primes(10000)

profile `mult_by` function when `primes()` is executed

In [18]:
%lprun -f mult_by primes(10000)

* `Line #`: The line number in the code
* `Hits`: The number of times that line was executed
* `Time`: The total amount of time spent executing the line in the timer’s units
* `Per Hit`: The average amount of time spent executing the line once in the timer’s unit
* `% Time`: The percentage of time spent on that line relative to the total amount of recorded time spent in the function
* `Line Contents`: The actual source code of the line

line_profiler generates profiles based on the lines in a program.

Line based profiling can provide more information about where a program is performing badly.